In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


base_url = "https://www.cantaringles.com/category/ingles-medio/page/{}/"

musicas_links = []

# Percorrer as páginas da categoria
for page in range(0, 200):  
    url = base_url.format(page)
    print(f"\n🔍 Acessando página da categoria: {url}")
    
    response = requests.get(url)
    if response.status_code != 200:
        print(f"⚠️ Erro ao acessar {url}")
        continue

    soup = BeautifulSoup(response.text, "html.parser")
    

    timeline = soup.find("ul", class_="timeline")
    
    if not timeline:
        print(f"⚠️ Estrutura <ul class='timeline'> não encontrada na página {url}")
        continue


    for link in timeline.find_all("a", href=True):
        href = link["href"]
        if href.startswith("https://www.cantaringles.com/") and href not in musicas_links:
            musicas_links.append(href)

    print(f"🎵 Encontrados {len(musicas_links)} links de músicas até agora.")
    time.sleep(2)  # Pequeno delay para evitar sobrecarga no servidor


lista_tabelas = []


def extrair_texto_formatado(td):
    soup = BeautifulSoup(str(td), "html.parser")

    # Primeiro o sublinhado (<u>)
    for underline in soup.find_all("u"):
        texto = underline.get_text()
        underline.clear()
        underline.append(f"{texto}̜")  # meia-lua abaixo

    # Depois os <span> com estilo
    for span in soup.find_all("span", style=True):
        estilo = span["style"]
        texto = span.get_text()

        if "color: #ff0000" in estilo:  # Vermelho → ponto/agudo abaixo
            span.clear()
            span.append(f"{texto}̗")  # agudo abaixo

        elif "color: #888888" in estilo:  # Cinza → underline
            span.clear()
            span.append(f"{texto}̳")  # duplo underline

        elif "color: #0000ff" in estilo:  # Azul → til abaixo
            span.clear()
            span.append(f"{texto}̰")  # til abaixo

    return soup.get_text(separator="").strip()


# acessa cada música e extrair as tabelas corretamente
for idx, music_url in enumerate(musicas_links):
    print(f"\n🎼 [{idx+1}/{len(musicas_links)}] Acessando música: {music_url}")

    response = requests.get(music_url)
    if response.status_code != 200:
        print(f"Erro ao acessar {music_url}")
        continue

    soup = BeautifulSoup(response.text, "html.parser")

    tabelas_div = soup.find_all("div", class_="su-table su-table-alternate")

    if not tabelas_div:
        print(f"Nenhuma tabela encontrada em {music_url}")
        continue

    print(f"{len(tabelas_div)} tabela(s) encontrada(s) na página.")

    for tabela_div in tabelas_div:
        tabela = tabela_div.find("table")  
        if not tabela:
            continue

        linhas = tabela.find_all("tr")
        tabela_extraida = []

        for linha in linhas:
            colunas = linha.find_all("td")
            if len(colunas) == 4:  # Certificando que tem 4 colunas (Número, Fonética, Inglês, Português)
                linha_extraida = [
                    extrair_texto_formatado(colunas[1]),  # Transcrição Fonética
                    extrair_texto_formatado(colunas[2]),  # Frase em Inglês
                    extrair_texto_formatado(colunas[3])   # Tradução para Português
                ]
                tabela_extraida.append(linha_extraida)

        # Criar um DataFrame para cada tabela extraída
        if tabela_extraida:  
            df = pd.DataFrame(tabela_extraida, columns=["Phonetic", "English", "Portuguese"])
            df["music_url"] = music_url  
            lista_tabelas.append(df)

    time.sleep(2)  # Pequeno delay entre requisições


df_final = pd.concat(lista_tabelas, ignore_index=True)


df_final.to_csv("transcricao_fonetica_formatada_final.csv", index=False)


🔍 Acessando página da categoria: https://www.cantaringles.com/category/ingles-medio/page/0/
🎵 Encontrados 1 links de músicas até agora.

🔍 Acessando página da categoria: https://www.cantaringles.com/category/ingles-medio/page/1/
🎵 Encontrados 1 links de músicas até agora.

🔍 Acessando página da categoria: https://www.cantaringles.com/category/ingles-medio/page/2/
🎵 Encontrados 21 links de músicas até agora.

🔍 Acessando página da categoria: https://www.cantaringles.com/category/ingles-medio/page/3/
🎵 Encontrados 71 links de músicas até agora.

🔍 Acessando página da categoria: https://www.cantaringles.com/category/ingles-medio/page/4/
🎵 Encontrados 84 links de músicas até agora.

🔍 Acessando página da categoria: https://www.cantaringles.com/category/ingles-medio/page/5/
🎵 Encontrados 85 links de músicas até agora.

🔍 Acessando página da categoria: https://www.cantaringles.com/category/ingles-medio/page/6/
🎵 Encontrados 135 links de músicas até agora.

🔍 Acessando página da categoria: h

In [23]:
df_final

,Phonetic,English,Portuguese,music_url
0,bikóz ai lâv̳,Because I love,Porque eu amo,https://www.cantaringles.com/because-i-love-so...
1,ól dei ai s̗ênk̳ óv̳ iú,All day I think of you,Todo dia eu penso em você,https://www.cantaringles.com/because-i-love-so...
2,iú fêl mai maind̳ fûl óv̳ dr̰ím̜s,You fill my mind full of dreams,Você enche minha mente de sonhos,https://www.cantaringles.com/because-i-love-so...
3,bikóz ai lâv̳,Because I love,Porque eu amo,https://www.cantaringles.com/because-i-love-so...
4,,,,https://www.cantaringles.com/because-i-love-so...
5,ai si z̗ét̳ gâr̰l,I see that girl,Eu vejo aquela garota,https://www.cantaringles.com/because-i-love-so...
6,z̗ou xis far̰ âuêi,Though she’s far away,Embora ela esteja longe,https://www.cantaringles.com/because-i-love-so...
7,xi fiâlz mai maind̳ fûl óv̳ dr̰ím̜s,She fills my mind full of dreams,Ela enche minha mente cheia de sonhos,https://www.cantaringles.com/because-i-love-so...
8,bikóz ai lâv̳,Because I love,Porque eu amo,https://www.cantaringles.com/because-i-love-so...
9,,,,https://www.cantaringles.com/because-i-love-so...


In [31]:
df_final.to_csv('ws_transcritor_database_v7.csv', index=False)